<B>Transcribe voice to text</B>

In [47]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
import json
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
speech_key, service_region = os.getenv('SPEECH_SERVICE_KEY'), os.getenv('SPEECH_SERVICE_REGION')

In [49]:
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

In [50]:
all_results = []
def conversation_transcriber_session_started(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')

def conversation_transcriber_recognition_canceled(evt: speechsdk.SessionEventArgs):
    print('SessionCanceled event')

def conversation_transcriber_session_stopped(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def conversation_transcriber_transcribed(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print('Text={}\n'.format(evt.result.text))
        print('Speaker ID={}\n'.format(evt.result.speaker_id))
        all_results.append(evt.result.text)    
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('NOMATCH: Speech could not be TRANSCRIBED: {}\n'.format(evt.result.no_match_details))


In [53]:
def speech_recognize_from_large_file(audio_file):
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the conversation transcriber
    conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed)
    conversation_transcriber.session_started.connect(conversation_transcriber_session_started)
    conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped)
    conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled)
    # stop transcribing on either session stopped or canceled events
    conversation_transcriber.session_stopped.connect(stop_cb)
    conversation_transcriber.canceled.connect(stop_cb)

    conversation_transcriber.start_transcribing_async()

    # Waits for completion.
    while not done:
        time.sleep(.5)

    conversation_transcriber.stop_transcribing_async()

    transcript_file_path = "./bronze_transcript.txt"
    save_to_text_file(all_results, transcript_file_path)
    print("Transcript saved to: " + transcript_file_path)

In [24]:
#Save to text file
def save_to_text_file(text_array, file_path):
    with open(file_path, 'w') as f:
        # Write each element of the text array to the file
        for text in text_array:
            f.write(text + '\n')


In [55]:
if __name__ == "__main__":
    audio_file = "./BillGates_2010.wav"

    try:
        speech_recognize_from_large_file(audio_file)
        print(all_results)
    except Exception as err:
        print("Encountered exception. {}".format(err))

SessionStarted event
TRANSCRIBED:
Text=

Speaker ID=Unknown

TRANSCRIBED:
Text=

Speaker ID=Unknown

TRANSCRIBED:
Text=I'm going to talk today about energy and climate, and that might seem a bit surprising because my full time work at the Foundation is mostly about vaccines and seeds, about the things that we need to invent and deliver to help the poorest 2 billion live better lives.

Speaker ID=Guest-1

TRANSCRIBED:
Text=But energy and climate are extremely important to these people, in fact more important than to anyone else on the planet. The climate getting worse means that many years their crops won't grow, there'll be too much rain, not enough rain. Things will change in ways that their fragile environment simply can't support. And that leads to starvation, that leads to uncertainty, it leads to unrest. So the the climate changes will be terrible for them.

Speaker ID=Guest-1

TRANSCRIBED:
Text=Also, the price of energy is very important to them. In fact, if you could pick just o

<B>Summary the call</B> 

In [46]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import os

In [47]:
def sample_abstractive_summarization(documents) -> None:
    # [START abstract_summary]
    endpoint = os.environ["AZURE_LANGUAGE_ENDPOINT"]
    key = os.environ["AZURE_LANGUAGE_KEY"]

    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )
    
    poller = text_analytics_client.begin_abstract_summary(documents)
    abstract_summary_results = poller.result()
    for result in abstract_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            [print(f"{summary.text}\n") for summary in result.summaries]
            
            #Save the summary results to a file
            summary_file_path = "./bronze_summary.txt"
            with open(summary_file_path, 'w') as f:
                for summary in result.summaries:
                    f.write(summary.text + '\n')
                print("Summary saved to: " + summary_file_path)

        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))
    # [END abstract_summary]

In [51]:
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip('\n') for line in f.readlines()]
        separator = ', '
        result = [separator.join(lines)]
        return result

In [52]:
if __name__ == "__main__":
    file_path = "./bronze_transcript.txt"
    documents = read_text_file(file_path)
    print(documents)
    try:
        sample_abstractive_summarization(documents) 
    except Exception as err:
            print("Encountered exception. {}".format(err))

[", , I'm going to talk today about energy and climate, and that might seem a bit surprising because my full time work at the Foundation is mostly about vaccines and seeds, about the things that we need to invent and deliver to help the poorest 2 billion live better lives., But energy and climate are extremely important to these people, in fact more important than to anyone else on the planet. The climate getting worse means that many years their crops won't grow, there'll be too much rain, not enough rain. Things will change in ways that their fragile environment simply can't support. And that leads to starvation, that leads to uncertainty, it leads to unrest. So the the climate changes will be terrible for them., Also, the price of energy is very important to them. In fact, if you could pick just one thing to lower the price of to reduce poverty by far, you would pick energy now. The price of energy has come down over time., Really advanced civilization is based on advances in in ene